In [2]:
"""Visualization utilies."""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def show_points(points, title):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_title(title)
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points[:, 0], points[:, 2], points[:, 1])

def compare_points(points1, points2):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_xlim3d([-2, 2])
    ax.set_ylim3d([-2, 2])
    ax.set_zlim3d([0, 4])
    ax.scatter(points1[:, 0], points1[:, 2], points1[:, 1])
    ax.scatter(points2[:, 0], points2[:, 2], points2[:, 1])

In [12]:
# Load the canonical-space object point clouds
import trimesh
import collada
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

def fps_downsample(points, number_of_points_to_sample):
  selected_points = np.zeros((number_of_points_to_sample, 3))
  dist = np.ones(points.shape[0]) * np.inf # distance to the selected set
  for i in tqdm(range(number_of_points_to_sample)):
      # pick the point with max dist
      idx = np.argmax(dist)
      selected_points[i] = points[idx]
      dist_ = ((points - selected_points[i]) ** 2).sum(-1)
      dist = np.minimum(dist, dist_)
  return selected_points

object_models_file = "models/objects_v1.csv"
object_models_info = pd.read_csv(object_models_file)
object_models_name = object_models_info["object"].to_list()
object_models_location = object_models_info["location"].to_list()
object_models_num = len(object_models_name)


object_models = {}
for object_name, object_mesh_dir in zip(object_models_name, object_models_location):
  object_mesh_path = os.path.join(object_mesh_dir, "visual_meshes", "visual.dae")
  # Collada-based loading
  object_mesh = trimesh.exchange.dae.load_collada(object_mesh_path)
  mesh_name = object_mesh['graph'][0]['geometry']
  object_models[object_name] = object_mesh['geometry'][mesh_name]['vertices']
  # Trimesh-based loading
  # object_mesh = trimesh.load(object_mesh_path, force='mesh')
  # object_models[object_name] = object_mesh.vertices
  # print(object_mesh)

# object_models with key=objectName, value=objectPointCloud
# print(object_models_info)
print(object_models["jenga"])
print(len(object_models["jenga"]))
# print(np.unique(object_models["jenga"]))
# print(len(object_models))







[[-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998  0.02499997 -0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [-0.07499998 -0.02499997  0.01499998]
 [-0.07499998  0.02499997  0.01499998]
 [-0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997 -0.01499998]
 [ 0.07499998  0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [ 0.07499998 -0.02499997  0.01499998]
 [-0.07499998 -0.02499997

In [4]:
"""Data loading helpers"""
import numpy as np
import json
import matplotlib.pyplot as plt
import os
from PIL import Image
import pickle
import utils
import open3d as o3d
# # Helper function for the reconstruction of the target point cloud
# rgb = np.array(Image.open(rgb_files[0])) / 255   # convert 0-255 to 0-1
# depth = np.array(Image.open(depth_files[0])) / 1000   # convert from mm to m
# label = np.array(Image.open(label_files[0]))
# meta = load_pickle(meta_files[0])
# intrinsic = meta['intrinsic']
# z = depth
# v, u = np.indices(z.shape)
# uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
# points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
# # print(points_viewer.shape)
# # print(points_viewer.shape[0]*points_viewer.shape[1])
# # print(points_viewer)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def dump_json(sample, path):
  with open(path, 'w') as fp:
    json.dump(sample, fp)
  return 0

def load_json(path):
  f = open(path)
  data = json.load(f)
  return data

def get_point_cloud(depth, intrinsic):
  z = depth
  v, u = np.indices(z.shape)
  uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
  points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
  return points_viewer

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3dvis(pts):
    o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
    return 0

def get_object_point_cloud(test_image_label, object_id, test_depth, intrinsic):
  # print(np.where(test_image_label==object_id))
  # print(test_image_label[327][654])
  test_image_label[np.where(test_image_label==object_id)] = 255
  # print(test_image_label[327][654])
  # print(np.where(test_image_label==255))
  test_image_label[np.where(test_image_label!=255)] = 0
  test_image_label[np.where(test_image_label==255)] = 1
  test_object_depth = test_depth * test_image_label
  test_pcd_target = get_point_cloud(test_object_depth, intrinsic)
  # (H, W, dim) = test_pcd_target.shape
  # filter out target object point cloud
  # print(test_pcd_target.shape)
  # print(test_pcd_target)
  # print((test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0))
  test_pcd_target = test_pcd_target.reshape(-1, test_pcd_target.shape[-1]) # reshape to (H*W, 3)
  test_pcd_target = test_pcd_target[(test_pcd_target[:,0]!=0)|(test_pcd_target[:,1]!=0)|(test_pcd_target[:,2]!=0)]
  return test_pcd_target


def get_meta(meta_path):
  return load_pickle(meta_path)

def get_depth(depth_path):
  return (np.array(Image.open(depth_path))/1000)

def get_label(label_path):
  return np.array(Image.open(label_path))


def dump_json(sample, path):
  with open(path, 'w') as fp:
    json.dump(sample, fp)
  return 0

def load_json(path):
  f = open(path)
  data = json.load(f)
  return data

def get_point_cloud(depth, intrinsic):
  z = depth
  v, u = np.indices(z.shape)
  uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
  points_viewer = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
  return points_viewer


def get_meta(meta_path):
  return load_pickle(meta_path)

def get_depth(depth_path):
  return (np.array(Image.open(depth_path))/1000)

def get_label(label_path):
  return np.array(Image.open(label_path))


"""Metric and visualization."""

def compute_rre(R_est: np.ndarray, R_gt: np.ndarray):
    """Compute the relative rotation error (geodesic distance of rotation)."""
    assert R_est.shape == (3, 3), 'R_est: expected shape (3, 3), received shape {}.'.format(R_est.shape)
    assert R_gt.shape == (3, 3), 'R_gt: expected shape (3, 3), received shape {}.'.format(R_gt.shape)
    # relative rotation error (RRE)
    # Rotational degree loss (not objective of optimization)
    rre = np.arccos(np.clip(0.5 * (np.trace(R_est.T @ R_gt) - 1), -1.0, 1.0))
    return rre


def compute_rte(t_est: np.ndarray, t_gt: np.ndarray):
    assert t_est.shape == (3,), 't_est: expected shape (3,), received shape {}.'.format(t_est.shape)
    assert t_gt.shape == (3,), 't_gt: expected shape (3,), received shape {}.'.format(t_gt.shape)
    # relative translation error (RTE)
    rte = np.linalg.norm(t_est - t_gt) # Resembling MSE loss
    return rte




In [110]:
"""Visualization on comparing between train pose-transformed point clouds."""
import open3d as o3d
import numpy as np
import time
import copy

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def get_o3d_icp_tensor(source_pcd, target_pcd):
  # T = icp(source_pcd, target_pcd, 150)
  source_pcd_o3d, target_pcd_o3d = o3d.t.geometry.PointCloud(), o3d.t.geometry.PointCloud()
  # print(source_pcd)
  source_pcd_o3d.point.positions = o3d.core.Tensor(source_pcd, dtype=o3d.core.Dtype.Float64)
  # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  target_pcd_o3d.point.positions = o3d.core.Tensor(target_pcd, dtype=o3d.core.Dtype.Float64)
  # Search distance for Nearest Neighbour Search [Hybrid-Search is used].
  max_correspondence_distance = 1
  init = o3d.core.Tensor.eye(4, o3d.core.Dtype.Float64)
  # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
  treg = o3d.cpu.pybind.t.pipelines.registration
  estimation = treg.TransformationEstimationPointToPoint()
  callback_after_iteration = lambda updated_result_dict : print("Iteration Index: {}, Fitness: {}, Inlier RMSE: {},".format(
    updated_result_dict["iteration_index"].item(), \
    updated_result_dict["fitness"].item(), \
    updated_result_dict["inlier_rmse"].item())) \

  # Convergence-Criteria for Vanilla ICP
  criteria = treg.ICPConvergenceCriteria(relative_fitness=0.0000000000001,
                      relative_rmse=0.0000000000001,
                      max_iteration=100)

  # Down-sampling voxel-size.
  # voxel_size = 0.0016625
  # voxel_size = 0.003125
  voxel_size = 0.00625
  # voxel_size = 0.0125

  # Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
  save_loss_log = True
  s = time.time()

  # voxel_size = 0.05  # means 5cm for this dataset
  # source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
  # result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
  # init = o3d.core.Tensor(result_ransac.transformation, dtype=o3d.core.Dtype.Float64)

  registration_icp = treg.icp(source_pcd_o3d, target_pcd_o3d, max_correspondence_distance, init, estimation, criteria, voxel_size, callback_after_iteration)
  icp_time = time.time() - s
  T = registration_icp.transformation
  T = T.numpy()
  return T


def get_o3d_icp(source_pcd, target_pcd, init=np.eye(4)):
  # T = icp(source_pcd, target_pcd, 150)
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  # print(source_pcd)
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  # source_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  # print(source_pcd_o3d)
  # print(target_pcd_o3d)
  # target_pcd_o3d.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(knn=50))
  # init = np.random.rand(4,4)
  # init = -1 + init * 2
  # init[-1] = np.zeros(4)
  # print(init)
  # init[0,0] = 2
  T = o3d.pipelines.registration.registration_icp( \
        source_pcd_o3d, target_pcd_o3d, 10, init, \
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), \
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.00000001, \
                                 relative_rmse=0.00000001, \
                                 max_iteration=5000))
  # T = icp(source_pcd, target_pcd)
  # print(T.inlier_rmse)
  # print(T.fitness)
  # print(T.correspondence_set)
  return T

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    # pts = pcd.points
    # size = len(pts)
    # if (size >= 1000):
    #   pcd_down = pcd.farthest_point_down_sample(size//2)
    # else: 
    #   pcd_down = pcd
    pcd_down = pcd

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size, source, target):
    print(":: Load two point clouds and disturb initial pose.")

    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
  distance_threshold = voxel_size * 1.5
  # print(":: RANSAC registration on downsampled point clouds.")
  # print("   Since the downsampling voxel size is %.3f," % voxel_size)
  # print("   we use a liberal distance threshold %.3f." % distance_threshold)
  result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
      source_down, target_down, source_fpfh, target_fpfh, True,
      distance_threshold,
      o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
      3, [
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
              0.9),
          o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
              distance_threshold)
      ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
  return result

def execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
  distance_threshold = voxel_size * 0.5
  print(":: Apply fast global registration with distance threshold %.3f" \
          % distance_threshold)
  result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
      source_down, target_down, source_fpfh, target_fpfh,
      o3d.pipelines.registration.FastGlobalRegistrationOption(
          maximum_correspondence_distance=distance_threshold))
  return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size, init):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

def get_o3d_icp_with_global_registration(source_pcd, target_pcd, voxel_size=10, init_random_time=10):
  source_pcd_o3d, target_pcd_o3d = o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
  source_pcd_o3d.points = o3d.utility.Vector3dVector(source_pcd)
  target_pcd_o3d.points = o3d.utility.Vector3dVector(target_pcd)
  source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, source_pcd_o3d, target_pcd_o3d)
  print(source, target)
  print(source_down, target_down)

  # perform multiple RANSAC inits, and choose the best amongest them
  Trans, rmse = [], []
  for i in tqdm(range(init_random_time)):
    result_ransac = execute_global_registration(source_down, target_down, \
                      source_fpfh, target_fpfh, \
                      voxel_size)
    # print("result_ransac: ")
    # print(result_ransac)
    # print(result_ransac.transformation.shape)
    # o3dvis([getTransPcd(source_down.points, result_ransac.transformation), target_down.points])
    # result_icp = refine_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, result_ransac.transformation)
    # TODO: Fetch the best initialization as the result
    result_icp = get_o3d_icp(source_down.points, target_down.points, result_ransac.transformation)
    # print("result_icp:")
    # print(result_icp)
    Trans.append(result_icp.transformation)
    rmse.append(result_icp.inlier_rmse)
    # draw_registration_result(source_down, target_down, result_icp.transformation)
  print("rmse list:")
  print(rmse)
  return Trans[np.argmin(rmse)]

def pts_to_o3d_pcd(pts):
  """Transform to o3d pcd"""
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(pts)
  return pcd

def o3dvis(pts):
  o3d.visualization.draw_geometries([pts_to_o3d_pcd(pt) for pt in pts])
  return 0

def getTransPcd(source_pcd, T):
  return source_pcd @ T[:3, :3].T + T[:3, 3]


# Define the data point to visualize
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-4", 0, [35, 39, 48, 51, 58], 58
rgb_file = "datas/1-1-4_color_kinect.png"
depth_file = "datas/1-1-4_depth_kinect.png"
label_file = "datas/1-1-4_label_kinect.png"
meta_file = "datas/1-1-4_meta.pkl"


# Load scene meta info
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
print(meta_vis.keys())
intrinsic = meta_vis["intrinsic"]
extrinsic = meta_vis["extrinsic"]
inv_extrinsic = np.linalg.inv(extrinsic)
gt_T_world = meta_vis["poses_world"][vis_id]
gt_T = extrinsic @ gt_T_world
# print(gt_T.shape)
gt_T = meta_vis["poses_world"][vis_id]
# Fetch source point cloud from model dictionary
# print(object_models)


# Reconstruct source and target point clouds (in camera frame)
print(object_name)
object_name = object_models_name[vis_id]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
vis_pcd_target = get_object_point_cloud(get_label(label_file), \
                    vis_id, \
                    get_depth(depth_file), \
                    intrinsic)

# o3dvis([vis_pcd_source])

print("src and tg len:")
print(len(vis_pcd_target), len(vis_pcd_source))


# Perform ICP
# source_pcd, target_pcd = vis_pcd_target, vis_pcd_source
source_pcd, target_pcd = vis_pcd_source, vis_pcd_target
# source_pcd = np.hstack((source_pcd, np.ones((len(source_pcd), 1))))
# target_pcd = np.hstack((target_pcd, np.ones((len(target_pcd), 1))))
# source_pcd, target_pcd = (source_pcd @ np.linalg.inv(extrinsic).T), (target_pcd @ np.linalg.inv(extrinsic).T)
# source_pcd = np.array([x[:-1]/x[-1] for x in source_pcd])
# target_pcd = np.array([x[:-1]/x[-1] for x in target_pcd])
# print(source_pcd.shape, target_pcd.shape)
# o3dvis([source_pcd, target_pcd])
T = get_o3d_icp_with_global_registration(source_pcd, target_pcd, voxel_size=5)
# T = get_o3d_icp_tensor(source_pcd, target_pcd)
# T = get_o3d_icp(source_pcd, target_pcd) # should use T.transformation to get transformation matrix!
# T = icp(source_pcd, target_pcd, 100)
T_world = inv_extrinsic @ T
TrEstiPcd = source_pcd @ T[:3, :3].T + T[:3, 3]
TrGtPcd = source_pcd @ gt_T[:3, :3].T + gt_T[:3, 3]

o3dvis([TrEstiPcd, target_pcd])


print(len(TrEstiPcd), len(TrGtPcd))
# print(TrEstiPcd)
# print(TrGtPcd)
# o3d.visualization.draw_geometries([pts_to_o3d_pcd(TrEstiPcd), pts_to_o3d_pcd(TrGtPcd)])

# T = np.eye(4)
print(source_pcd.shape)
print(target_pcd.shape)
print("GT T:")
print(gt_T)
print("GT T world:")
print(gt_T_world)
print("pred T:")
print(T)
print("pred T world:")
print(T_world)
# rre = np.rad2deg(compute_rre(T[:3, :3], gt_T[:3, :3]))
# rte = compute_rte(T[:3, 3], gt_T[:3, 3])
rre = np.rad2deg(compute_rre(T_world[:3, :3], gt_T_world[:3, :3]))
rte = compute_rte(T_world[:3, 3], gt_T_world[:3, 3])
print(f"rre={rre}, rte={rte}")
# show_points(target_pcd, "Target pcd")
# show_points(source_pcd, "Source pcd")
# show_points(TrEstiPcd, "Transformed source pcd")
# show_points(TrGtPcd, "GT Transformed source pcd")






dict_keys(['poses_world', 'extents', 'scales', 'object_ids', 'object_names', 'extrinsic', 'intrinsic'])
pudding_box
src and tg len:
4135 49152
:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 5.000.
:: Estimate normal with search radius 10.000.
:: Compute FPFH feature with search radius 25.000.
:: Downsample with a voxel size 5.000.
:: Estimate normal with search radius 10.000.
:: Compute FPFH feature with search radius 25.000.
PointCloud with 49152 points. PointCloud with 4135 points.
PointCloud with 49152 points. PointCloud with 4135 points.


100%|██████████| 10/10 [00:28<00:00,  2.84s/it]


rmse list:
[0.01176805705359216, 0.010612464069852225, 0.010887726200242871, 0.010612464667375062, 0.011477113912545744, 0.011768057053592108, 0.01144558727610238, 0.0105925372291997, 0.010735323891631022, 0.011434162143288778]
49152 49152
(49152, 3)
(4135, 3)
GT T:
[[ 0.00145066  0.99998546  0.00521667  0.27790332]
 [-0.9999827   0.00148046 -0.00571105 -0.06234347]
 [-0.00571869 -0.00520829  0.9999701   0.02262924]
 [ 0.          0.          0.          1.        ]]
GT T world:
[[ 0.00145066  0.99998546  0.00521667  0.27790332]
 [-0.9999827   0.00148046 -0.00571105 -0.06234347]
 [-0.00571869 -0.00520829  0.9999701   0.02262924]
 [ 0.          0.          0.          1.        ]]
pred T:
[[ 0.99574795 -0.08321756  0.03950768 -0.10587012]
 [ 0.07676105  0.98665715  0.14358069  0.12900343]
 [-0.05092897 -0.13993753  0.9888497   0.62512751]
 [ 0.          0.          0.          1.        ]]
pred T world:
[[-0.03562217  0.86214034 -0.5054157   0.28400288]
 [ 0.99530219 -0.01495223 -0.0956

In [6]:
"""Visualize given pcd"""
train_vis_varianct, train_vis_index, object_ids, vis_id = "1-1-1", 0, [35, 39, 48, 51, 58], 58
object_name = object_models_name[vis_id]
meta_vis = get_meta(meta_file)
scales = meta_vis["scales"]
vis_pcd_source = object_models[object_name]
vis_pcd_source = vis_pcd_source * scales[vis_id]
o3d.visualization.draw_geometries([pts_to_o3d_pcd(vis_pcd_source)])


